<a href="https://colab.research.google.com/github/pachterlab/kb_docs/blob/main/docs/source/translated/notebooks/virus_detection_sc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Viral proteins in single-cell RNA seq data

[Open in Google Colab](https://colab.research.google.com/github/pachterlab/kb_docs/blob/main/docs/source/translated/notebooks/virus_detection_sc.ipynb)

In this tutorial, we will align single-cell RNA sequencing data collected from Dengue virus (DENV)-infected CEM.NK$^{R}$ cells as well as PBMC data collected from a patient suffering from DENV (during and after infection) to viral RdRP protein sequences. Let's see if we can detect DENV RdRP-like sequences as expected.
  
**Data references:**  
RNA seq data:  
https://www.nature.com/articles/s41598-020-65939-5  
https://www.ncbi.nlm.nih.gov/Traces/study/?acc=PRJNA613041&o=acc_s%3Aa  
PalmDB viral protein reference database:  
https://www.nature.com/articles/s41586-021-04332-2  
  
Written by: Laura Luebbert (last updated: 11/13/2024)

In [ ]:
# Number of threads to use during alignments
threads = 2

## Install software

In [ ]:
!pip install -q kb_python ffq gget anndata==0.10.9

## Download raw sequencing data
Here, we will download the data from DENV1 infected CEM.NK^R cells (SRR11321027) and the uninfected control (SRR11321026) as well as PBMC data collected from a patient experiencing a DENV infection at fever day -2 (SRR11321526) and 180 (baseline post-infection) (SRR11321528):

In [ ]:
import json

## Get the FTP download links for raw data using ffq and store the results in a json file
!ffq SRR11321027 SRR11321026 SRR11321526 SRR11321528 \
    --ftp \
    -o ffq.json

## Load ffq output
f = open("ffq.json")
data_json = json.load(f)
f.close()

## Download raw data using FTP links fetched by ffq
# NOTE: To decrease the runtime and required disk space of this example notebook,
# we will only download the first [gb_to_download] GB of each fastq file
gb_to_download = 5

# Convert GB to bytes
max_bytes = gb_to_download * 1073741824
# Download data
for dataset in data_json:
    url = dataset["url"]
    # (Remove the `-r 0-$max_bytes` argument to download the complete fastq files)
    !curl -r 0-$max_bytes -O $url

## Download the protein reference
In this case, we are using an optimized version of the [PalmDB database](https://github.com/ababaian/palmdb). The database was optimized for the detection of viral proteins from RNA sequencing data as described in [this manuscript](https://www.biorxiv.org/content/10.1101/2023.12.11.571168) and the files are stored in the [accompanying repository](https://github.com/pachterlab/LSCHWCP_2023/tree/main).

In [ ]:
# Download the ID to taxonomy mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/ID_to_taxonomy_mapping.csv
# Download the customized transcripts to gene mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
# Download the RdRP amino acid sequences
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa

## Build a reference index from the viral protein sequences and mask host (here, human) sequences

In addition to building a reference index from the protein sequences, we also want to mask host (in this case, human) sequences.

Here, we are downloading a precomputed PalmDB reference index for use with `kb` in which the human genome and transcriptome were masked. You can find all available precomputed reference indeces [here](https://github.com/pachterlab/LSCHWCP_2023/tree/main/precomputed_refs).

In [ ]:
# Download precomputed index with masked human genome and transcriptome
!wget https://data.caltech.edu/records/sh33z-hrx98/files/palmdb_human_dlist_cdna_dna.idx?download=1
!mv palmdb_human_dlist_cdna_dna.idx?download=1 palmdb_human_dlist_cdna_dna.idx

**Alternatively, you can compute the PalmDB reference index yourself (and define a different host species):**

The [`--aa` argument](https://kallisto.readthedocs.io/en/latest/translated/pseudoalignment.html) tells `kb` that this is an amino acid reference.

The [`--d-list` argument](https://kallisto.readthedocs.io/en/latest/index/index_generation.html#the-d-list) is the path to the host genome/transcriptome. These sequences will be masked in the index. Here, we are using gget to fetch the latest human genome and transcriptome from Ensembl.

We are using `--workflow custom` here since we do not have a .gtf file for the PalmDB fasta file.

Building the index will take some time (~20 min), since the human genome is quite large.

In [ ]:
# # Build a PalmDB reference index and mask the human genome and transcriptome from scratch

# # Download the human reference genome and transcriptome from Ensembl
# # Replace 'homo_sapiens' here and the genome/transcriptome file names below if the host is a different species
# !gget ref -w cdna,dna -d homo_sapiens

# # Concatenate human genome and transcriptome into one file
# !cat Homo_sapiens.GRCh38.cdna.all.fa.gz Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz > Homo_sapiens.GRCh38.cdna_dna.fa.gz

# %%time
# !kb ref \
#   --workflow custom \
#   --aa \
#   --d-list Homo_sapiens.GRCh38.cdna_dna.fa.gz \
#   -t $threads \
#   -i palmdb_human_dlist_cdna_dna.idx \
#   palmdb_rdrp_seqs.fa

## Align data using kallisto translated search

Create a batch.txt file so we can run all fastq files simultaneously (to learn more about batch files, see Box 7 in the [Protocols paper](https://www.biorxiv.org/content/10.1101/2023.11.21.568164v2.full.pdf)) (to keep track of which cell barcodes belonged to which SRR file, we will activate the `--batch-barcodes` option in the alignment step below):

In [ ]:
%%time
import numpy as np

samples = [
    'SRR11321526',
    'SRR11321528',
    'SRR11321027',
    'SRR11321026'
    ]

# NOTE: To decrease the runtime of this example notebook,
# we will only align the top [n_seq_to_keep] sequences in each fastq file
n_seq_to_keep = 20000000
print(f"Number of reads kept per fastq file: {n_seq_to_keep:,}")

with open("batch.txt", "w") as batch_file:
  for sample in samples:
    R1 = f"/content/{sample}_1.fastq.gz"
    R2 = f"/content/{sample}_2.fastq.gz"

    # Shorten fastq files (skip this step during a real analysis)
    n_rows_to_keep = n_seq_to_keep * 4
    R1_short = R1.split(".fastq.gz")[0] + "_short.fastq"
    R2_short = R2.split(".fastq.gz")[0] + "_short.fastq"
    !zcat $R1 | head -$n_rows_to_keep > $R1_short && gzip $R1_short
    !zcat $R2 | head -$n_rows_to_keep > $R2_short && gzip $R2_short

    # Write batch file in the following format:
    # sample_name \t R1_filepath \t R2_filepath
    batch_file.write(sample + "\t" + R1_short + ".gz" + "\t" + R2_short + ".gz" + "\n")

#### Align fastqs using kallisto translated search:  
The [`-x` argument](https://kallisto.readthedocs.io/en/latest/sc/technologies.html) tells `kb` where to find the barcodes and UMIs in the data. Here, we set it to `10xv2` because this data was generated "using the Chromium Single-Cell 5′ Reagent version 2 kit."

In [ ]:
%%time

!kb count \
    --verbose \
    --aa \
    -x 10xv2 \
    -t $threads \
    -i palmdb_human_dlist_cdna_dna.idx \
    -g palmdb_clustered_t2g.txt \
    --h5ad \
    -o kb_output \
    --batch-barcodes \
    batch.txt

## Load the generated count matrix

In [ ]:
import anndata
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.colors
%config InlineBackend.figure_format='retina'

def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

In [ ]:
# Open count matrix (AnnData object in h5ad format)
adata = anndata.read_h5ad("kb_output/counts_unfiltered/adata.h5ad")
adata

In [ ]:
# 'barcode' here includes batch/sample barcode (first 16 characters) + cell barcode
adata.obs

Add column showing which SRR file each cell came from using the sample barcodes:

In [ ]:
# Load the sample/batch barcodes
b_file = open("kb_output/matrix.sample.barcodes")
barcodes = b_file.read().splitlines()
b_file.close()

# Load the sample/batch names
s_file = open("kb_output/matrix.cells")
samples = s_file.read().splitlines()
s_file.close()

# Create dataframe that maps sample barcodes to sample names
bc2sample_df = pd.DataFrame()
bc2sample_df["sample_barcode"] = barcodes
bc2sample_df["srr"] = samples

# Add sample name to AnnData object
adata.obs["barcode"] = adata.obs.index.values
adata.obs["sample_barcode"] = [bc[:-16] for bc in adata.obs.index.values]
adata.obs["cell_barcode"] = [bc[-16:] for bc in adata.obs.index.values]
adata.obs = adata.obs.merge(bc2sample_df, on="sample_barcode", how="left").set_index("cell_barcode", drop=False)
adata.obs

## Plot counts for Dengue virus (DENV) in each sample

In [ ]:
# Load the PalmDB virus ID to virus taxonomy mapping
u_tax_csv = "ID_to_taxonomy_mapping.csv"
tax_df = pd.read_csv(u_tax_csv)

# Find the reference IDs for Dengue virus RdRPs
# Note: Only the rep_ID (representative ID) will occur in the count matrix
tax_df[tax_df["species"].str.contains("Dengue virus")]

Create bar plot showing raw counts for DENV:

In [ ]:
fig, ax = plt.subplots(figsize=(6, 7))
fontsize = 16
width = 0.75

samples = [
    'SRR11321526',
    'SRR11321528',
    'SRR11321027',
    'SRR11321026'
    ]

# Label corresponding to each SRR library (samples)
x_labels = [
    'Patient PBMC\n(fever day -2)',
    'Patient PBMC\n(post-infection control)',
    'DENV-infected CEM.NK$^R$',
    'Uninfected\nControl (CEM.NK$^R$)'
    ]

# Target virus IDs to plot (here, Dengue virus)
target_virus = "Dengue virus"
target_ids = tax_df[tax_df["species"].str.contains(target_virus)]["rep_ID"].unique()

counts = []
labels = samples
for sample in samples:
    counts.append(adata.X[adata.obs["srr"].str.contains(sample), adata.var.index.isin(target_ids)].sum())

x = np.arange(len(labels))

ax.bar(x, counts, width=width, color="#003049", edgecolor="black")

ax.set_yscale("symlog")
ax.set_ylabel("kallisto (raw counts for Dengue virus)", fontsize=fontsize)
# ax.set_xlabel("Sample", fontsize=fontsize)

ax.set_xticks(x, x_labels, rotation=45, ha="right")

ax.tick_params(axis="both", labelsize=fontsize)

ax.grid(True, which="both", color="lightgray", ls="--", lw=1)
ax.set_axisbelow(True)

# Save figure
plt.savefig("denv_rdrp_count.png", dpi=300, bbox_inches="tight")

fig.show()

In [ ]:
# These counts are low since we only aligned a small subset of the data
counts